## Rover Project Notebook
* Write new functions (or modify existing ones) to report and map out detections of obstacles and rock samples (yellow rocks)
* Populate the `process_image()` function with the appropriate steps/functions to go from a raw image to a worldmap.
* Run the cell that calls `process_image()` using `moviepy` functions to create video output
* Once you have mapping working, move on to modifying `perception.py` and `decision.py` to allow your rover to navigate and map in autonomous mode!

**Get code highlighting in the markdown cells.**

In [ ]:
%%HTML
<style> code {background-color : orange !important;} </style>

** Configure the Notebook plotting style. **

In [ ]:
%matplotlib inline

** Import the necessary modules. **

In [ ]:
import numpy as np
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import scipy.misc                    # For saving images as needed
import glob                          # For reading in a list of images from a folder
import imageio
import pandas as pd
from moviepy.editor import ImageSequenceClip

In [ ]:
# local code
import sys
sys.path.append("./code")
import nd_functions as ndf
import nd_classes as ndc

** Read in and display a random image from the `training_dataset` folder. **

In [ ]:
path = './training_dataset/IMG/*'

# make a list of the frames saved during training mode
img_list = glob.glob(path)
print("Total of %d images in the list."%(len(img_list)))

# Grab a random image from the list
idx = np.random.randint(0, len(img_list)-1)

# read in the picked immage
image = mpimg.imread(img_list[idx])

# print image size
print(image.shape, image.shape[0], image.shape[1])

# print some statistic
print(image.min(), image.max(), image.mean())

# show the image
plt.imshow(image)

# save the warped image
scipy.misc.imsave('./output/picked_example.jpg', image)

** Read in data for calibration. Display example grid and rock sample calibration images. Grid is needed for perspective transform and the rock image for creating a new color selection that identifies these samples so that can be picked up during autonomous navigation. **

In [ ]:
# pointer to image files
example_grid = './calibration_images/example_grid1.jpg'
example_rock = './calibration_images/example_rock1.jpg'

# load image from file pointer
grid_img = mpimg.imread(example_grid)
rock_img = mpimg.imread(example_rock)

# plot images
fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.imshow(grid_img)
plt.subplot(122)
plt.imshow(rock_img)

** Select the rock by thresholding it **

In [ ]:
#select the rock from the environment
rock_map = ndf.find_rocks(rock_img)

fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.imshow(rock_img)
plt.subplot(122)
plt.imshow(rock_map, cmap='gray')

** Create warped immage. **

In [ ]:
# These source and destination points are defined to warp the image to a grid where each 10x10 pixel square represents 
# 1 square meter

# identify the coordinates in the image of the grid. 
source = np.float32([[14, 140], 
                     [301 ,140], 
                     [200, 96], 
                     [118, 96]
                    ])

# The destination box will be 2*dst_size on each side
dst_size = 5 
# Set a bottom offset to account for the fact that the bottom of the image is not the position of the rover but a bit in front of it
bottom_offset = 6

# identify the destination box
destination = np.float32([[image.shape[1]/2 - dst_size, image.shape[0] - bottom_offset],
                          [image.shape[1]/2 + dst_size, image.shape[0] - bottom_offset],
                          [image.shape[1]/2 + dst_size, image.shape[0] - 2*dst_size - bottom_offset], 
                          [image.shape[1]/2 - dst_size, image.shape[0] - 2*dst_size - bottom_offset],
                        ])

# create warp immage and related mask
warped, mask = ndf.perspect_transform(grid_img, source, destination)

# save the warped image
scipy.misc.imsave('./output/warped_example.jpg', warped)

fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.imshow(warped)
plt.subplot(122)
plt.imshow(mask, cmap='gray')

** Identify ground pixels only for navigable terrain and obstacles. **

In [ ]:
# Using threshold of RGB > 160 as it does a nice job of identifying ground pixels only
nav_map = ndf.color_thresh(warped)

# create the obstacles map (inverting the nav_map mainly)
obstacles_map = np.absolute(np.float32(nav_map) - 1) * mask

#plt.imshow(nav_map, cmap='gray')
scipy.misc.imsave('./output/warped_threshed.jpg', nav_map*255)

fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.imshow(obstacles_map, cmap='gray')
plt.subplot(122)
plt.imshow(nav_map, cmap='gray')

** Coordintate Transformations **

In [ ]:
# Calculate pixel values in rover-centric coords. Since the thresh is in black and white, the trasformation will be on all the 
# pixesls in the white part. The returning valuea are 2 array x,y of all the pixels in white.... or highliting the navigable part.
xpix, ypix = ndf.rover_coords(nav_map)

# calculate distance/angle to all pixels. pixels are give as 2 arrays x,y.  
dist, angles = ndf.to_polar_coords(xpix, ypix)

# calculate mean angle from the set of angles. This correspondes to move in the meadle of the navigable terrain.
mean_dir = np.mean(angles)

# Do some plotting
fig = plt.figure(figsize=(12,9))
plt.subplot(221)
plt.imshow(image)

plt.subplot(222)
plt.imshow(warped)

plt.subplot(223)
plt.imshow(nav_map, cmap='gray')

plt.subplot(224)
plt.plot(xpix, ypix, '.')
plt.ylim(-160, 160)
plt.xlim(0, 160)

# draw an vector to indicate direction into the picture
arrow_length = 100
x_arrow = arrow_length * np.cos(mean_dir)
y_arrow = arrow_length * np.sin(mean_dir)
plt.arrow(0, 0, x_arrow, y_arrow, color='red', zorder=2, head_width=10, width=2)

** Read training dataset in pandas dataframe. **

In [ ]:
# create data frame
df = pd.read_csv('./training_dataset/robot_log.csv', delimiter=';', decimal=',')      

# check some results
df.head()

** Read in ground truth map and create a 3-channel image with it. **

In [ ]:
ground_truth = mpimg.imread('./calibration_images/map_bw.png')
print(ground_truth.shape)
ground_truth_3d = np.dstack((ground_truth*0, ground_truth*255, ground_truth*0)).astype(np.float)

fig = plt.figure(figsize=(12,3))
plt.subplot(121)
plt.imshow(ground_truth_3d)
plt.subplot(122)
plt.imshow(ground_truth, cmap='gray')

** Create a data bucket **

In [ ]:
# Create data object (Databucket) containing the training frames
data = ndc.Databucket(df, ground_truth_3d)

** Make a video from processed image data. **  

In [ ]:
# create a clip from a list of images in the Databucket
clip = ImageSequenceClip(data.images, fps=25) 

#  need to use the clip.fx method so that it can accept other arguments, clip.fl_image does nota ccept extra arguments
new_clip = clip.fx(ndf.img_transformation, data, source, destination)

# Define pathname to save the output video
output = './output/training_mapping.mp4'
%time new_clip.write_videofile(output, audio=False)

** inline video player **

In [ ]:
from IPython.display import HTML
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(output))